In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3

engine = sqlite3.connect('example.db')
def rsq(quary:str) -> pd.DataFrame:
    return pd.read_sql_query(quary, engine)

In [59]:
df = pd.read_csv('./annual_aqi_by_county_2018.csv')
df.to_sql('annual_aqi', engine, if_exists='replace', index=False)

1029

In [4]:
rsq("""-- Athlete count by country and region
SELECT reg.region
  , reg.country
  , COUNT(DISTINCT ath.athlete_id) AS no_athletes -- Athletes can compete in multiple events
FROM athletes AS ath
JOIN oregions AS reg
  ON reg.olympic_cc = ath.country_code
GROUP BY reg.region, reg.country
ORDER BY no_athletes;""")

,Region,Country,no_athletes
0,Africa,Equatorial Guinea,1
1,Australia and Oceania,Tuvalu,1
2,Africa,Chad,2
3,Africa,Liberia,2
4,Africa,Mauritania,2
...,...,...,...
202,Europe,Russia,497
203,Europe,France,499
204,North America,Canada,526
205,Europe,Germany,569


In [8]:
rsq("""SELECT reg.region, reg.country
  , COUNT(DISTINCT ath.athlete_id) AS no_athletes
FROM regions reg
LEFT JOIN athletes ath
  ON reg.olympic_cc = ath.country_code
GROUP BY reg.region, reg.country
ORDER BY no_athletes DESC;""")

,Region,Country,no_athletes
0,North America,USA,777
1,Europe,Germany,569
2,North America,Canada,526
3,Europe,France,499
4,Europe,Russia,497
...,...,...,...
206,Australia and Oceania,Tuvalu,1
207,None,Unknown,0
208,Asia,Kuwait,0
209,Asia,North Borneo,0


In [12]:
rsq("""SELECT reg.region, reg.country
  , COUNT(DISTINCT ath.athlete_id) AS no_athletes
FROM regions reg
left JOIN athletes ath
  ON ath.country_code = reg.olympic_cc
GROUP BY reg.region, reg.country
ORDER BY no_athletes DESC;""")

,Region,Country,no_athletes
0,North America,USA,777
1,Europe,Germany,569
2,North America,Canada,526
3,Europe,France,499
4,Europe,Russia,497
...,...,...,...
206,Australia and Oceania,Tuvalu,1
207,None,Unknown,0
208,Asia,Kuwait,0
209,Asia,North Borneo,0


In [13]:
rsq("""SELECT reg.region, reg.country
  , COUNT(DISTINCT ath.athlete_id) AS no_athletes
FROM regions reg
INNER JOIN athletes ath
  ON ath.country_code = reg.olympic_cc
GROUP BY reg.region, reg.country
ORDER BY no_athletes DESC;""")

,Region,Country,no_athletes
0,North America,USA,777
1,Europe,Germany,569
2,North America,Canada,526
3,Europe,France,499
4,Europe,Russia,497
...,...,...,...
202,Africa,Swaziland,2
203,Asia,Bhutan,2
204,Australia and Oceania,Nauru,2
205,Africa,Equatorial Guinea,1


In [14]:
rsq("""SELECT reg.region
  , ath.season
  , COUNT(DISTINCT ath.athlete_id) AS no_athletes
  , COUNT(DISTINCT reg.olympic_cc) AS no_countries
  , COUNT(DISTINCT ath.athlete_id)/COUNT(DISTINCT reg.olympic_cc) AS athletes_per_country
FROM athletes ath
inner JOIN oregions reg
  ON ath.country_code = reg.olympic_cc
GROUP BY reg.region, ath.season -- Group by region and season
order by reg.region,athletes_per_country ;""")

,Region,Season,no_athletes,no_countries,athletes_per_country
0,None,Winter,1,1,1
1,None,Summer,19,2,9
2,Africa,Winter,5,3,1
3,Africa,Summer,990,54,18
4,Asia,Winter,338,22,15
5,Asia,Summer,2138,47,45
6,Australia and Oceania,Winter,76,4,19
7,Australia and Oceania,Summer,740,18,41
8,Europe,Winter,1847,44,41
9,Europe,Summer,4779,45,106


In [28]:
rsq("""select a.*,b.Country
    from athletes a
    join oclimate b
    on a.country_code = b.Olympic_CC;""").to_sql('athletes', engine, if_exists='replace', index=False)

18147

In [29]:
rsq("""-- Countries cold enough for snow year-round
SELECT country_code, country, COUNT(DISTINCT athlete_id) AS winter_athletes -- Athletes can compete in multiple events
FROM athletes
WHERE country_code IN (SELECT olympic_cc FROM oclimate WHERE temp_annual < 0)
AND season = 'Winter'
GROUP BY country_code, country;""")

,Country_Code,country,winter_athletes
0,CAN,Canada,217
1,MGL,Mongolia,2
2,RUS,Russia,213


In [32]:
rsq("""WITH south_cte AS (SELECT region, ROUND(AVG(temp_06),2) AS avg_winter_temp, ROUND(AVG(precip_06),2) AS avg_winter_precip FROM oclimate WHERE region IN ('Africa','South America','Australia and Oceania') GROUP BY region)

SELECT south.region, south.avg_winter_temp, south.avg_winter_precip, COUNT(DISTINCT ath.athlete_id) as athlete_count FROM south_cte as south INNER JOIN athletes ath ON south.region = ath.region AND ath.season = 'Winter' GROUP BY south.region, south.avg_winter_temp, south.avg_winter_precip ORDER BY south.avg_winter_temp;""")

,region,avg_winter_temp,avg_winter_precip,athlete_count
0,South America,18.50,156.23,31
1,Australia and Oceania,19.87,137.92,75
2,Africa,23.99,73.98,5


In [35]:
rsq("""select * from oclimate;""").to_sql('climate', engine, if_exists='replace', index=False)

171

In [36]:
rsq("""
    with athletes_wint as (select * from athletes where season = 'Winter')
    -- Climate by country with Olympian athletes
SELECT country
  , temp_06
  , precip_06
FROM climate
WHERE region = 'Africa'
AND olympic_cc IN (SELECT Distinct country_code FROM athletes_wint)
ORDER BY temp_06;""")

,country,temp_06,precip_06
0,Zimbabwe,15.794618,3.666232
1,Morocco,22.220897,8.181283
2,Togo,26.007222,169.054445


In [41]:
rsq("""select * from athletes where season = 'Winter'""").to_sql('athletes_wint',engine, if_exists='replace', index=False)

4846

In [45]:
rsq("""    SELECT olympic_cc
      , country
      , temp_06
      , precip_06
    FROM climate
    WHERE region = 'Africa'""").to_sql('countries_cte', engine, if_exists='replace', index=False)

51

In [46]:
rsq("""  
SELECT DISTINCT cte.country
  , cte.temp_06
  , cte.precip_06
FROM athletes_wint AS wint
INNER JOIN countries_cte AS cte
  ON wint.country_code = cte.olympic_cc
ORDER BY temp_06;""")

,country,temp_06,precip_06
0,Zimbabwe,15.794618,3.666232
1,Morocco,22.220897,8.181283
2,Togo,26.007222,169.054445


In [50]:
rsq("""-- Create a temp table of Canadians

    SELECT *
    FROM athletes
    WHERE country_code = 'CAN'
    AND season = 'Winter'; -- The table has both summer and winter athletes""").to_sql("athletes_recent", engine, if_exists='replace', index=False)

359

In [51]:
rsq("""

-- Find the most popular sport
SELECT sport
  , COUNT(DISTINCT athlete_id) as no_athletes
FROM canadians
GROUP BY sport 
ORDER BY no_athletes DESC;""")

,Sport,no_athletes
0,Ice Hockey,44
1,Freestyle Skiing,26
2,Snowboarding,24
3,Bobsleigh,18
4,Speed Skating,17
5,Figure Skating,17
6,Alpine Skiing,15
7,Cross Country Skiing,13
8,Short Track Speed Skating,9
9,Luge,8


In [53]:
rsq("""SELECT DISTINCT o.region, a.country_code, o.country
   FROM athletes a
   INNER JOIN oregions o
      ON a.country_code = o.olympic_cc;""").to_sql("countries", engine, if_exists='replace', index=False)

169

In [57]:
def rsq(quary:str) -> pd.DataFrame:
    result = pd.read_sql_query(quary, engine)
    if result.empty:
        return pd.DataFrame(columns=result.columns)
    else:
        return result
pd.read_sql_query("ANALYZE countries; -- Collect the statistics",engine)

TypeError: 'NoneType' object is not iterable

In [54]:
rsq("""

SELECT COUNT(*) FROM countries; -- Count the entries""")

,COUNT(*)
0,169
